In [2]:
import whisperx as whisper
model=whisper.load_model("tiny")
result = model.transcribe("../audio/fold.wav",language='en',temperature=0.7,initial_prompt="prompt hun sir",condition_on_previous_text=True)


c:\Users\Ashhad\anaconda3\envs\p_39\lib\site-packages\whisperx\transcribe.py:83: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [3]:
import json
jj=result['segments']
with open("fold.json","w",encoding='utf-8') as f:
    json.dump(jj,f)

In [6]:
import json
with open("fold.json","r",encoding='utf-8') as f:
    cont=json.load(f)

In [9]:
import torch
if torch.cuda.is_available():
    device="gpu"
else:
    device="cpu"

In [29]:
model_a, metadata = whisper.load_align_model(language_code='en',device=device)

# align whisper output
result_aligned = whisper.align(cont, model_a, metadata, "../audio/fold.wav",device=device)

# print(result_aligned["segments"]) # after alignment
# print(result_aligned["word_segments"]) # after alignment

In [30]:
char_segments=[]
word_segments=[]

for x in range(len(result_aligned['segments'])):
    word_segments.append({"word-segments":result_aligned['segments'][x]['word-segments'].to_dict(orient='records')})
    char_segments.append({"char-segments":result_aligned['segments'][x]['char-segments'].to_dict(orient='records')})

SyntaxError: unexpected EOF while parsing (1498383660.py, line 6)

In [28]:
result_aligned

{'segments': [{'start': 0.6626506024096386,
   'end': 2.5301204819277108,
   'text': " today I'm going to show you how to fold a shirt.",
   'word-segments':                    start       end     score  segment-text-start  \
   subsegment-idx                                                     
   0               0.662651  0.943775  0.884123                   1   
   0               0.963855  1.024096  0.000061                   7   
   0               1.365462  1.465863  0.121185                  11   
   0               1.485944  1.526104  0.146264                  17   
   0               1.566265  1.686747  0.358255                  20   
   0               1.706827  1.807229  0.834148                  25   
   0               1.827309  1.927711  0.470288                  29   
   0               1.947791  2.008032  0.748700                  33   
   0               2.028112  2.188755  0.913051                  36   
   0               2.248996  2.269076  0.984014                 

In [42]:
import requests

url = "http://localhost:8000/whisperX/"
files = {'audio': ('../audio/fold.wav', open('../audio/fold.wav', 'rb'), 'audio/wav')}
data = {'transcription': 'srt', 'model_name': 'tiny','translate':False,
        "device":"cpu",
        "language":"en","patience":1.0,"temperature":1.0,
        "condition_on_previous/_text":False,"temperature_increment_on_fallback":0.2,
        "compression_ratio_threshold":2.4,"logprob_threshold":-1.0,"no_speech_threshold":0.6
        }

response = requests.post(url, files=files, data=data)

# if response.status_code == 200:
#     print(response.json())
# else:



#     print("Error:", response.content)
response.json()

{'segments_before_alignment': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.94,
   'text': " Today, I'm going to show you how to fold a shirt.",
   'tokens': [2692,
    11,
    286,
    478,
    516,
    281,
    855,
    291,
    577,
    281,
    4860,
    257,
    8336,
    13],
   'temperature': 1.0,
   'avg_logprob': -1.1965341741388493,
   'compression_ratio': 1.6666666666666667,
   'no_speech_prob': 0.07223737984895706,
   'seg-text': [" Today, I'm going to show you how to fold a shirt."]},
  {'id': 1,
   'seek': 0,
   'start': 5.14,
   'end': 7.48,
   'text': ' In under two second',
   'tokens': [682, 833, 732, 1150],
   'temperature': 1.0,
   'avg_logprob': -1.1965341741388493,
   'compression_ratio': 1.6666666666666667,
   'no_speech_prob': 0.07223737984895706,
   'seg-text': [' In under two second']},
  {'id': 2,
   'seek': 0,
   'start': 7.5200000000000005,
   'end': 11.6,
   'text': ' Start by taking a short sleeve shirt and 무�가ing it,',
   'tokens': [6481,
    538